In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

from deep_fib.sci_net import SCIBlockCfg, SCINet
from deep_fib.data import DeepFIBDataset, get_masks
from deep_fib.core import DeepFIBEngine

from utils.data import Marconi100Dataset, get_train_test_split
from utils.training import training_loop
from utils.summary import SummaryWriter

In [4]:
train, test = get_train_test_split(0.5, 42)


In [5]:
# train = train[:len(train)//30]
# test = test[:len(test)//10]
# len(train), len(test)


In [6]:
m_data_train = Marconi100Dataset(train, normalize=True)
m_data_test = Marconi100Dataset(test, normalize=True)


Loading:  18%|█████▊                           | 22/124 [00:10<00:51,  1.99it/s]

KeyboardInterrupt: 

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

horizon = 1024
stride = 1000
n_masks = 20

batch_size = 32
num_workers = 2

num_encoder_levels = 2

log_dir = "./trash"
lr = 1e-3
num_epochs = 3
step_size = 2

hidden = None
block_cfg = SCIBlockCfg(input_dim=460, hidden_size=4, kernel_size=3, dropout=0.5,)

anomaly_threshold = 0.7


cuda


In [2]:
dataset_train = DeepFIBDataset(
    m_data_train, horizon=horizon, stride=stride
)
dataset_test = DeepFIBDataset(m_data_test, horizon=horizon, stride=stride)

masks = get_masks(horizon, n_masks).float()

len(dataset_train), len(dataset_test), masks.size()


NameError: name 'DeepFIBDataset' is not defined

In [15]:
train_loader = DataLoader(
    dataset_train,
    batch_size,
    shuffle=True,
    num_workers=num_workers,
    persistent_workers=(num_workers != 0),
)
test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
    num_workers=num_workers,
    persistent_workers=(num_workers != 0),
)
len(train_loader), len(test_loader)


(1940, 100)

In [14]:
model = SCINet(
    output_len=horizon,
    input_len=horizon,
    num_encoder_levels=num_encoder_levels,
    hidden_decoder_sizes=hidden,
    block_config=block_cfg,
).float()

engine = DeepFIBEngine(anomaly_threshold, masks)

optim = Adam(model.parameters(), lr=lr)
lr_sched = StepLR(optim, step_size)

with SummaryWriter(log_dir) as writer:
    training_loop(
        model=model,
        engine=engine,
        num_epochs=num_epochs,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        device=device,
        optimizer=optim,
        lr_scheduler=lr_sched,
        writer=writer,
        save_path=log_dir + "/models",
    )


KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=$log_dir